In [1]:
import praw
import pandas as pd
import datetime
import time
from psaw import PushshiftAPI


# Pushshift API

This API can search for more than 1000 data and specific time range


In [2]:
# Search for more 1000 data

api  = PushshiftAPI()
search_date = datetime.datetime(2010,1,1)
start_epoch = int(search_date.timestamp())
subreddit   = 'investing'
data = api.search_submissions(after=start_epoch,
                             subreddit = subreddit,
                             sort_type = 'score',
                             sort = 'desc',
                             filter=['author', 'title', 'subreddit' ,'score', 'selftext'],
                             limit=50000)


In [3]:
df = pd.DataFrame(list(data))
df = df[['created_utc','author','score','title','selftext']]
for i in range(len(df)):
    df['created_utc'].iloc[i] = time.strftime("%Y-%m-%d",time.gmtime(df['created_utc'].iloc[i]))

D:\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
df.columns=['Date','Name','Score','Title','Content']
df = df.set_index('Date')
df.head()

,Name,Score,Title,Content
Date,,,,
2019-10-14,ChocolateTsar,14102,Bernie Sanders unveils plan to raise corporate...,Democratic presidential hopeful Bernie Sanders...
2017-04-11,Cfdms009,13731,United Airlines stock down over 5% premarket t...,http://www.marketwatch.com/story/uniteds-stock...
2018-08-03,04housemat,10351,"If in 2001, you bought $399 of Apple stock ins...",
2019-03-13,helpmycareerout,10169,"According to the Federal Reserve, the average ...",https://turbo.intuit.com/blog/real-money-talk/...
2017-08-14,troycatalano,8103,Merck CEO resigns from Trump's American Manufa...,https://www.cnbc.com/2017/08/14/merck-ceo-resi...


In [5]:
df

,Name,Score,Title,Content
Date,,,,
2019-10-14,ChocolateTsar,14102,Bernie Sanders unveils plan to raise corporate...,Democratic presidential hopeful Bernie Sanders...
2017-04-11,Cfdms009,13731,United Airlines stock down over 5% premarket t...,http://www.marketwatch.com/story/uniteds-stock...
2018-08-03,04housemat,10351,"If in 2001, you bought $399 of Apple stock ins...",
2019-03-13,helpmycareerout,10169,"According to the Federal Reserve, the average ...",https://turbo.intuit.com/blog/real-money-talk/...
2017-08-14,troycatalano,8103,Merck CEO resigns from Trump's American Manufa...,https://www.cnbc.com/2017/08/14/merck-ceo-resi...
...,...,...,...,...
2010-01-01,[deleted],1,What are the best investments in ALGAE?,[removed]
2010-01-01,MoneyEnergy,1,Best Performing Canadian Stocks and ETFs of 19...,
2009-12-31,CWFEU,1,Is the Chinese Growth Story Heading for a Bubble?,


In [16]:
df.to_csv('./Reddit Data/'+ subreddit + '_' +datetime.datetime.strftime(search_date,"%Y-%m-%d") + '.csv')

In [17]:
len(df)

15092

# Praw API
This is Reddit offical API can search specific word but less than 1000 data

In [9]:
reddit = praw.Reddit(client_id='-RIN1YYCUr4Beg',
                     client_secret='2HifNYi9IFLXRDgGPh27IzbmMg8',
                     user_agent='crawler')

print(reddit.read_only)

True


In [10]:
data = []

# Range 

subreddit = reddit.subreddit("investing")
keywords  = "fund"
limits    = 1000
time_filter = 'year'

# Import API
# https://praw.readthedocs.io/en/latest/code_overview/models/submission.html
# sort='relevance','hot','top','new','comments'
# syntax='lucene' , 'cloudsearch' , 'plain'
# time_filter='year'

for submission in subreddit.search(keywords, sort='relevance', syntax='lucene', time_filter=time_filter, limit=limits):
    date = time.gmtime(submission.created_utc)
    data.append([time.strftime("%Y-%m-%d",date),submission.author,submission.score,submission.title,submission.selftext])
    
df = pd.DataFrame(data,columns=['Date','Name','Score','Title','Content'])
df = df.set_index('Date')
df = df.dropna()
df.head()

,Name,Score,Title,Content
Date,,,,


In [11]:
df.to_csv('./Reddit Data/keywords_'+ time_filter + '_'+ keywords +'.csv')

In [12]:
data = []

# Range 
time_now = datetime.datetime.now()

subreddit = reddit.subreddit("investing")
keywords  = "hot"
limits    = 1000
time_filter = 'year'

for submission in subreddit.hot(limit=None):
    date = time.gmtime(submission.created_utc)
    data.append([time.strftime("%Y-%m-%d",date),submission.author,submission.score,submission.title,submission.selftext])
    
df = pd.DataFrame(data,columns=['Date','Name','Score','Title','Content'])
df = df.set_index('Date')
df

,Name,Score,Title,Content
Date,,,,
2019-09-01,MasterCookSwag,271,Formal posting guidelines for political topics...,"Alright everyone, it looks like we had pretty ..."
2020-06-03,AutoModerator,11,Daily Advice Thread - All basic help or advice...,"If your question is ""I have $10,000, what do I..."
2020-06-03,jayjayy123,446,"Choosing a stock is like choosing your gf, if ...",What are some of your favourite picks?
2020-06-03,SD987,335,List of stocks still down more than 40%,I'm trying to compile a list of stocks that st...
2020-06-03,achicomp,1066,May ADP employment report: -2.76 million vs ex...,Absolutely monster job report beat. Stock futu...
...,...,...,...,...
2020-05-21,rustcohlerian4444,0,Gold is such a snake oil.,So... we should start with saying Iam a huge B...
2020-05-20,chikva1,2,"Money supply is growing, stocks prices are jus...",\n\nBeginning from mid February the total mon...
2020-05-20,jay2tall,1,Thoughts on DHT,My dividend is pending @ $.35 a share. Price k...


In [ ]:
df.to_csv('./Reddit Data/keywords_'+ time_filter + '_'+ keywords +'.csv')

# Push Shift API

The Json version for query data

In [ ]:
data = []

subreddit_name = 'investing'
days = '500d'

url = 'https://api.pushshift.io/reddit/submission/search/?after='+days+'&sort_type=created_utc&sort=asc&subreddit='+subreddit_name
df = pd.read_json(url,orient='split')
df

In [ ]:
dfa = df[['created_utc','author','score','title','selftext']]
dfa